In [4]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 18.5 MB/s eta 0:00:0000:0100:01


Import necessary files

In [5]:
import os
import zipfile
import shutil
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
import numpy as np
import model as model

Download relevant datasets from https://drivingstereo-dataset.github.io/ and place them in the data/downloaded folder.

In [6]:
# datasets to download

left_images = "https://drive.google.com/drive/folders/1VjxCnOeEIGKF6GqDqrpWvDTM4keTWAC_?usp=sharing"
right_images = "https://drive.google.com/drive/folders/1VjxCnOeEIGKF6GqDqrpWvDTM4keTWAC_?usp=sharing"
disparity_maps = "https://drive.google.com/drive/folders/1m6gJawtGrmrpO6DhC6jrrrDd6WMWOJCS?usp=sharing"
depth_maps = "https://drive.google.com/drive/folders/1bKtlM-1JaG1YYR6yBDKFOH6r8yoUyDNm?usp=sharing"

# calibration files
train = "https://drive.google.com/drive/folders/19e-qWaOXS5L2pLtYv7T_cJX2c6a4lOaf?usp=sharing"
test = "https://drive.google.com/drive/folders/1cP6zOsHJUh7u3m4Sufq23ccoZmDIJrHp?usp=sharing"

if not os.path.exists("data"):
    os.mkdir("data")


Download files, extract and setup dataset

In [13]:




folder_names = ["left_images", "right_images", "disparity_maps", "depth_maps"]

def extract_and_sort_files(source_folder, target_folder_dict):
    def filename_is_in_keys(filename, target_folder_dict):
        for keyword in target_folder_dict.keys():
            if keyword in filename:
                return target_folder_dict[keyword]
        return False

    # List all files in the source folder
    files = os.listdir(source_folder)
    
    os.makedirs("tmp", exist_ok=True)
    for file in files:
        # Construct full file path
        file_path = os.path.join(source_folder, file)
        
        # Check if the file is a zip file
        if file.endswith('.zip') and filename_is_in_keys(file, target_folder_dict):
            target_folder = filename_is_in_keys(file, target_folder_dict)
            tmp_folder = f"tmp/{target_folder}" 
            os.makedirs(tmp_folder, exist_ok=True)

            # # Open the zip file
            print(f'Extracting {file} to {tmp_folder}')
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                # Extract all files from the zip
                zip_ref.extractall(tmp_folder)
                # List zip files and extract them again
                for extracted_file in os.listdir(tmp_folder):
                    if os.path.isdir(os.path.join(tmp_folder, extracted_file)):
                        for extracted_file2 in os.listdir(os.path.join(tmp_folder, extracted_file)):
                            extracted_file_path = os.path.join(tmp_folder, extracted_file, extracted_file2)
                            if extracted_file2.endswith('.zip'):
                                with zipfile.ZipFile(extracted_file_path, 'r') as zip_ref:
                                    zip_ref.extractall(tmp_folder)
                                os.remove(extracted_file_path)
                        os.removedirs(os.path.join(tmp_folder, extracted_file))


# Example usage
source_folder = 'data/downloaded'
target_folder_dict = {
    'train-depth-map': 'data/train/depth_maps',
    'train-left-image': 'data/train/left_images',
    'train-right-image': 'data/train/right_images',
    'train-disparity-map': 'data/train/disparity_maps',
    # 'test-depth-map': 'data/test/depth_maps',
    # 'test-left-image': 'data/test/left_images',
    # 'test-right-image': 'data/test/right_images',
    # 'test-disparity-map': 'data/test/disparity_maps',
}

extract_and_sort_files(source_folder, target_folder_dict)


Extracting train-right-image-20240514T134932Z-019.zip to tmp/data/train/right_images
Extracting train-left-image-20240514T131122Z-004.zip to tmp/data/train/left_images
Extracting train-depth-map-20240514T184238Z-015.zip to tmp/data/train/depth_maps
Extracting train-right-image-20240514T134932Z-018.zip to tmp/data/train/right_images
Extracting train-left-image-20240514T131122Z-017.zip to tmp/data/train/left_images
Extracting train-left-image-20240514T131122Z-008.zip to tmp/data/train/left_images
Extracting train-disparity-map-20240514T202809Z-006.zip to tmp/data/train/disparity_maps
Extracting train-depth-map-20240514T184238Z-020.zip to tmp/data/train/depth_maps
Extracting train-left-image-20240514T131122Z-009.zip to tmp/data/train/left_images
Extracting train-right-image-20240514T134932Z-009.zip to tmp/data/train/right_images
Extracting train-disparity-map-20240514T202809Z-011.zip to tmp/data/train/disparity_maps
Extracting train-right-image-20240514T134932Z-014.zip to tmp/data/train/r

In [15]:
# Analyze the number of images, and the size of the images
import cv2
import numpy as np

def analyze_images(folder, samples=5):
    files = os.listdir(folder)
    print(f'Analyzing {len(files)} images in {folder}')
    for i in range(samples):
        file = np.random.choice(files)
        file_path = os.path.join(folder, file)
        image = cv2.imread(file_path)
        print(f'{file_path} - {image.shape}')
    
    print(f"Resolution of images in {folder}: {image.shape[0]}x{image.shape[1]}")
        

analyze_images('tmp/data/train/left_images')
analyze_images('tmp/data/train/right_images')
analyze_images('tmp/data/train/disparity_maps')
analyze_images('tmp/data/train/depth_maps')

Analyzing 174437 images in tmp/data/train/left_images
tmp/data/train/left_images/2018-10-26-15-24-18_2018-10-26-15-57-50-120.jpg - (400, 879, 3)
tmp/data/train/left_images/2018-10-15-11-43-36_2018-10-15-12-12-22-566.jpg - (400, 879, 3)
tmp/data/train/left_images/2018-07-16-15-18-53_2018-07-16-15-18-59-585.jpg - (400, 881, 3)
tmp/data/train/left_images/2018-10-23-13-59-11_2018-10-23-14-52-31-218.jpg - (400, 879, 3)
tmp/data/train/left_images/2018-10-23-08-34-04_2018-10-23-09-14-43-067.jpg - (400, 879, 3)
Resolution of images in tmp/data/train/left_images: 400x879
Analyzing 174437 images in tmp/data/train/right_images
tmp/data/train/right_images/2018-07-24-14-31-18_2018-07-24-15-11-04-830.jpg - (400, 881, 3)
tmp/data/train/right_images/2018-10-18-10-39-04_2018-10-18-11-21-26-176.jpg - (400, 879, 3)
tmp/data/train/right_images/2018-07-18-10-16-21_2018-07-18-10-45-52-984.jpg - (400, 881, 3)
tmp/data/train/right_images/2018-10-24-14-13-21_2018-10-24-14-23-52-961.jpg - (400, 879, 3)
tmp/data

In [7]:
# create a train loader that loads the images and disparity maps


class StereoDataset(Dataset):
    def __init__(self, left_images_folder, right_images_folder, disparity_maps_folder, transform=None):
        self.left_images_folder = left_images_folder
        self.right_images_folder = right_images_folder
        self.disparity_maps_folder = disparity_maps_folder
        self.transform = transform
        
        self.left_images = os.listdir(left_images_folder)
        self.right_images = os.listdir(right_images_folder)
        self.disparity_maps = os.listdir(disparity_maps_folder)
        
        self.left_images.sort()
        self.right_images.sort()
        self.disparity_maps.sort()

        # check that the number of images is the same
        print(f'Found {len(self.left_images)} left images')
        print(f'Found {len(self.right_images)} right images')
        print(f'Found {len(self.disparity_maps)} disparity maps')
        
        assert len(self.left_images) == len(self.right_images) == len(self.disparity_maps)

        
    def __len__(self):
        return len(self.left_images)
    
    def __getitem__(self, idx):
        left_image = cv2.imread(os.path.join(self.left_images_folder, self.left_images[idx]))
        right_image = cv2.imread(os.path.join(self.right_images_folder, self.right_images[idx]))
        disparity_map = cv2.imread(os.path.join(self.disparity_maps_folder, self.disparity_maps[idx]), cv2.IMREAD_GRAYSCALE)
        
        if self.transform:
            left_image = self.transform(left_image)
            right_image = self.transform(right_image)
            disparity_map = self.transform(disparity_map)
        
        return left_image, right_image, disparity_map

Load the dataset

In [8]:

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

dataset = StereoDataset('tmp/data/train/left_images', 'tmp/data/train/right_images', 'tmp/data/train/disparity_maps', transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

for left_image, right_image, disparity_map in dataloader:
    print(left_image.shape, right_image.shape, disparity_map.shape)
    break




Found 174437 left images
Found 174437 right images
Found 174437 disparity maps
torch.Size([4, 3, 256, 256]) torch.Size([4, 3, 256, 256]) torch.Size([4, 1, 256, 256])
